In [35]:
import pandas as pd
import yfinance as yf

In [36]:
def process_leveraged_data(tickers, leverage_scalars, portfolio_weights):
    """
    Download historical data for given tickers, simulate leveraged ETF data, 
    and calculate their weighted contributions to the portfolio.

    Parameters:
        tickers (list of str): List of ticker symbols (e.g., ["QQQ", "SPY"]).
        leverage_scalars (list of float): List of scalars for leveraged equity returns.
        portfolio_weights (list of float): List of portfolio weights for each ticker.

    Returns:
        pd.DataFrame: DataFrame with formatted columns for each ticker and weighted portfolio returns.
    """
    if len(tickers) != len(leverage_scalars) or len(tickers) != len(portfolio_weights):
        raise ValueError("The number of tickers, leverage scalars, and portfolio weights must match.")
    
    if not abs(sum(portfolio_weights) - 1.0) < 1e-6:
        raise ValueError("Portfolio weights must sum to 1.")

    result_df = pd.DataFrame()  # Initialize an empty DataFrame for results

    for ticker, scalar, weight in zip(tickers, leverage_scalars, portfolio_weights):
        print(f"Downloading data for {ticker}...")

        # Download the baseline data
        baseline_data = yf.download(ticker, progress=False)

        # Calculate daily returns
        baseline_data['Daily Return'] = baseline_data['Adj Close'].pct_change()

        # Simulate leveraged returns based on the scalar
        baseline_data['Leveraged Return'] = baseline_data['Daily Return'] * scalar
        baseline_data.loc[baseline_data.index[0], 'Leveraged Return'] = 0  # Ensure the first leveraged return is 0

        # Calculate simulated leveraged price
        baseline_data['Simulated Leveraged Price'] = (1 + baseline_data['Leveraged Return']).cumprod()
        baseline_data['Simulated Leveraged Price'] *= baseline_data['Adj Close'].iloc[0]  # Normalize to starting price

        # Weighted leveraged return
        baseline_data['Weighted Leveraged Return'] = baseline_data['Leveraged Return'] * weight

        # Rename columns to include the ticker symbol
        baseline_data = baseline_data.rename(
            columns={
                'Daily Return': f'DailyReturn_{ticker}',
                'Adj Close': f'AdjClose_{ticker}',
                'Leveraged Return': f'LeveragedReturn_{ticker}',
                'Simulated Leveraged Price': f'SimulatedLeveragedPrice_{ticker}',
                'Weighted Leveraged Return': f'WeightedLeveragedReturn_{ticker}'
            }
        )

        # Keep only relevant columns and reset index
        baseline_data = baseline_data[[f'DailyReturn_{ticker}', f'AdjClose_{ticker}', 
                                       f'LeveragedReturn_{ticker}', f'SimulatedLeveragedPrice_{ticker}', 
                                       f'WeightedLeveragedReturn_{ticker}']]
        baseline_data.reset_index(inplace=True)

        # Merge with the result DataFrame
        if result_df.empty:
            result_df = baseline_data
        else:
            result_df = pd.merge(result_df, baseline_data, on='Date', how='outer')

    # Calculate total portfolio returns
    weighted_columns = [col for col in result_df.columns if col.startswith('LeveragedReturn')]
    result_df['TotalPortfolioReturn'] = result_df[weighted_columns].sum(axis=1)

    return result_df

In [37]:
# Define inputs
tickers = ["QQQ", "SPY", "DIA"]
leverage_scalars = [3, 2, 1.5]  # Leverage scalars for each ticker
portfolio_weights = [0.5, 0.3, 0.2]  # Portfolio weights summing to 1.0

# Process the data
data = process_leveraged_data(tickers, leverage_scalars, portfolio_weights)
data.head()

,Date,DailyReturn_QQQ,AdjClose_QQQ,LeveragedReturn_QQQ,SimulatedLeveragedPrice_QQQ,WeightedLeveragedReturn_QQQ,DailyReturn_SPY,AdjClose_SPY,LeveragedReturn_SPY,SimulatedLeveragedPrice_SPY,WeightedLeveragedReturn_SPY,DailyReturn_DIA,AdjClose_DIA,LeveragedReturn_DIA,SimulatedLeveragedPrice_DIA,WeightedLeveragedReturn_DIA,TotalPortfolioReturn
0,1993-01-29,NaN,NaN,NaN,NaN,NaN,NaN,24.526093,0.000000,24.526093,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000
1,1993-02-01,NaN,NaN,NaN,NaN,NaN,0.007111,24.700504,0.014223,24.874916,0.004267,NaN,NaN,NaN,NaN,NaN,0.014223
2,1993-02-02,NaN,NaN,NaN,NaN,NaN,0.002120,24.752859,0.004239,24.980365,0.001272,NaN,NaN,NaN,NaN,NaN,0.004239
3,1993-02-03,NaN,NaN,NaN,NaN,NaN,0.010570,25.014500,0.021140,25.508456,0.006342,NaN,NaN,NaN,NaN,NaN,0.021140
4,1993-02-04,NaN,NaN,NaN,NaN,NaN,0.004184,25.119169,0.008369,25.721929,0.002511,NaN,NaN,NaN,NaN,NaN,0.008369
